<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=83db5c06f29408991e274bd647dedbc9d3f3b7d6d123dc2afd9c05758aa869c7
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'CLEAN', 'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND',
       'HDFCAMC', 'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC',
       'ITBEES', 'JMFINANCIL', 'JUBLFOOD', 'LUXIND', 'METROPOLIS',
       'NIACL', 'PAGEIND', 'PIDILITIND', 'PNB', 'RAJESHEXPO', 'RELAXO',
       'RELIANCE', 'ROSSARI', 'SUNTV', 'TCS', 'TEAMLEASE', 'UJJIVANSFB',
       'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 42
Investment:  2863855.0
Current:  2965992.0
Today PnL:  1199.0
Today PnL %:  0.04
Current PnL:  102137.0
Current PnL %:  3.57
Estimate:  5160755.0
Estimate PnL:  2296900.0
Estimate PnL %:  80.2


In [8]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,1302.55,ICICIGI.NS,1613.0,1292.25,1137.80,69.0,V40,ATH,78508.20,89875.95,89165.25,111297.0,0.80,14.48,41.76,27.28,2.74,40.0
38,38.35,UJJIVANSFB.NS,57.0,38.30,34.87,1350.0,V40N,ATH,47074.50,51772.50,51705.00,76950.0,0.13,9.98,63.46,53.48,1.64,16.0
6,7206.15,BAJFINANCE.NS,7901.0,7248.80,6602.15,24.0,V40,ATH,158451.60,172947.60,173971.20,189624.0,-0.59,9.15,19.67,10.52,5.53,51.0
16,2048.50,HDFCAMC.NS,3518.0,2104.90,1879.66,41.0,V40,ATH,77066.06,83988.50,86300.90,144238.0,-2.68,8.98,87.16,78.18,2.69,24.0
8,1650.05,BATAINDIA.NS,2173.0,1640.80,1523.62,100.0,V40,ATH,152362.00,165005.00,164080.00,217300.0,0.56,8.30,42.62,34.32,5.32,44.0
4,3316.10,ASIANPAINT.NS,3542.0,3318.70,3091.34,25.0,V40,ATH,77283.50,82902.50,82967.50,88550.0,-0.08,7.27,14.58,7.31,2.70,20.0
23,74.20,JMFINANCIL.NS,118.0,73.85,69.17,675.0,2T3Y,2T3Y,46689.75,50085.00,49848.75,79650.0,0.47,7.27,70.59,63.32,1.63,12.0
13,186.05,GICRE.NS,358.0,187.95,174.18,231.0,2T3Y,2T3Y,40235.58,42977.55,43416.45,82698.0,-1.01,6.81,105.53,98.72,1.40,10.0
11,3919.50,DMART.NS,5324.0,3986.90,3671.87,21.0,SS,ATH,77109.27,82309.50,83724.90,111804.0,-1.69,6.74,44.99,38.25,2.69,18.0
29,2671.25,PIDILITIND.NS,2902.0,2678.50,2504.06,30.0,V40,ATH,75121.80,80137.50,80355.00,87060.0,-0.27,6.68,15.89,9.21,2.62,18.0


In [9]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,1006.35,GLAND.NS,4316.0,1022.00,1225.87,55.0,2T3Y,2T3Y,67422.85,55349.25,56210.00,237380.0,-1.53,-17.91,252.08,269.99,2.35,-42.0
40,561.70,VALIANTORG.NS,1851.0,559.75,542.52,144.0,2T3Y,2T3Y,78122.88,80884.80,80604.00,266544.0,0.35,3.54,241.19,237.65,2.73,10.0
25,1555.55,LUXIND.NS,4546.0,1559.80,1493.72,51.0,V40N,2T3Y,76179.72,79333.05,79549.80,231846.0,-0.27,4.14,204.34,200.20,2.66,11.0
39,315.60,VAIBHAVGBL.NS,966.0,313.45,317.48,144.0,2T3Y,2T3Y,45717.12,45446.40,45136.80,139104.0,0.69,-0.59,204.27,204.86,1.60,-1.0
7,241.60,BANDHANBNK.NS,723.0,247.00,253.98,300.0,2T3Y,2T3Y,76194.00,72480.00,74100.00,216900.0,-2.19,-4.87,184.67,189.54,2.66,-13.0
27,120.55,NIACL.NS,317.0,122.60,117.07,390.0,TBD,ATH,45657.30,47014.50,47814.00,123630.0,-1.67,2.97,170.78,167.81,1.59,5.0
26,1456.10,METROPOLIS.NS,3440.0,1424.20,1365.98,22.0,NaN,ATH,30051.56,32034.20,31332.40,75680.0,2.24,6.60,151.83,145.23,1.05,7.0
37,2419.10,TEAMLEASE.NS,5281.0,2416.85,2279.93,21.0,V40N,2T3Y,47878.53,50801.10,50753.85,110901.0,0.09,6.10,131.63,125.53,1.67,10.0
1,1383.30,AAVAS.NS,3196.0,1364.20,1390.60,23.0,2T3Y,2T3Y,31983.80,31815.90,31376.60,73508.0,1.40,-0.52,129.83,130.35,1.12,-1.0
18,1466.15,INDIGOPNTS.NS,3112.0,1429.60,1407.73,124.0,V40N,2T3Y,174558.52,181802.60,177270.40,385888.0,2.56,4.15,121.07,116.92,6.10,25.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
4,3316.10,ASIANPAINT.NS,3542.0,3318.70,3091.34,25.0,V40,ATH,77283.50,82902.50,82967.50,88550.0,-0.08,7.27,14.58,7.31,2.70,20.0
29,2671.25,PIDILITIND.NS,2902.0,2678.50,2504.06,30.0,V40,ATH,75121.80,80137.50,80355.00,87060.0,-0.27,6.68,15.89,9.21,2.62,18.0
33,2564.30,RELIANCE.NS,2811.0,2557.10,2467.15,31.0,V40,ATH,76481.65,79493.30,79270.10,87141.0,0.28,3.94,13.94,10.00,2.67,11.0
6,7206.15,BAJFINANCE.NS,7901.0,7248.80,6602.15,24.0,V40,ATH,158451.60,172947.60,173971.20,189624.0,-0.59,9.15,19.67,10.52,5.53,51.0
36,3258.20,TCS.NS,3857.0,3227.70,3231.00,16.0,V40,ATH,51696.00,52131.20,51643.20,61712.0,0.94,0.84,19.37,18.53,1.81,2.0
5,1519.25,BAJAJFINSV.NS,1907.0,1521.05,1454.93,52.0,V40,ATH,75656.36,79001.00,79094.60,99164.0,-0.12,4.42,31.07,26.65,2.64,12.0
17,1302.55,ICICIGI.NS,1613.0,1292.25,1137.80,69.0,V40,ATH,78508.20,89875.95,89165.25,111297.0,0.80,14.48,41.76,27.28,2.74,40.0
22,30.39,ITBEES.NS,40.0,30.33,30.07,1675.0,SS,ATH,50367.25,50903.25,50802.75,67000.0,0.20,1.06,33.02,31.96,1.76,2.0
3,448.10,AMBUJACEM.NS,588.0,448.95,424.02,180.0,TBD,ATH,76323.60,80658.00,80811.00,105840.0,-0.19,5.68,38.67,32.99,2.67,15.0
8,1650.05,BATAINDIA.NS,2173.0,1640.80,1523.62,100.0,V40,ATH,152362.00,165005.00,164080.00,217300.0,0.56,8.30,42.62,34.32,5.32,44.0


In [11]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1466.15,INDIGOPNTS.NS,3112.0,1429.60,1407.73,124.0,V40N,2T3Y,174558.52,181802.60,177270.40,385888.0,2.56,4.15,121.07,116.92,6.10,25.0
6,7206.15,BAJFINANCE.NS,7901.0,7248.80,6602.15,24.0,V40,ATH,158451.60,172947.60,173971.20,189624.0,-0.59,9.15,19.67,10.52,5.53,51.0
8,1650.05,BATAINDIA.NS,2173.0,1640.80,1523.62,100.0,V40,ATH,152362.00,165005.00,164080.00,217300.0,0.56,8.30,42.62,34.32,5.32,44.0
28,38259.65,PAGEIND.NS,53549.0,38647.35,39325.03,3.0,V40,ATH,117975.09,114778.95,115942.05,160647.0,-1.00,-2.71,36.17,38.88,4.12,-11.0
14,4550.15,GILLETTE.NS,7529.0,4547.45,4475.85,21.0,V40,ATH,93992.85,95553.15,95496.45,158109.0,0.06,1.66,68.21,66.55,3.28,5.0
0,366.90,5PAISA.NS,553.0,356.10,347.06,228.0,V40N,ATH,79129.68,83653.20,81190.80,126084.0,3.03,5.72,59.34,53.62,2.76,16.0
17,1302.55,ICICIGI.NS,1613.0,1292.25,1137.80,69.0,V40,ATH,78508.20,89875.95,89165.25,111297.0,0.80,14.48,41.76,27.28,2.74,40.0
40,561.70,VALIANTORG.NS,1851.0,559.75,542.52,144.0,2T3Y,2T3Y,78122.88,80884.80,80604.00,266544.0,0.35,3.54,241.19,237.65,2.73,10.0
4,3316.10,ASIANPAINT.NS,3542.0,3318.70,3091.34,25.0,V40,ATH,77283.50,82902.50,82967.50,88550.0,-0.08,7.27,14.58,7.31,2.70,20.0
24,495.15,JUBLFOOD.NS,903.0,497.00,471.56,164.0,SS,ATH,77335.84,81204.60,81508.00,148092.0,-0.37,5.00,91.49,86.49,2.70,14.0


In [12]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [13]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,1006.35,GLAND.NS,4316.0,1022.00,1225.87,55.0,2T3Y,2T3Y,67422.85,55349.25,56210.00,237380.0,-1.53,-17.91,252.08,269.99,2.35,-42.0
7,241.60,BANDHANBNK.NS,723.0,247.00,253.98,300.0,2T3Y,2T3Y,76194.00,72480.00,74100.00,216900.0,-2.19,-4.87,184.67,189.54,2.66,-13.0
31,555.25,RAJESHEXPO.NS,949.0,564.30,624.78,48.0,V40N,ATH,29989.44,26652.00,27086.40,45552.0,-1.60,-11.13,51.89,63.02,1.05,-12.0
28,38259.65,PAGEIND.NS,53549.0,38647.35,39325.03,3.0,V40,ATH,117975.09,114778.95,115942.05,160647.0,-1.00,-2.71,36.17,38.88,4.12,-11.0
41,385.50,WIPRO.NS,713.0,382.50,388.07,195.0,2T3Y,2T3Y,75673.65,75172.50,74587.50,139035.0,0.78,-0.66,83.73,84.39,2.64,-2.0
19,1286.60,INDUSINDBK.NS,1965.0,1298.40,1301.47,36.0,SS,ATH,46852.92,46317.60,46742.40,70740.0,-0.91,-1.14,50.98,52.12,1.64,-2.0
10,1369.55,CLEAN.NS,2648.0,1363.40,1396.49,23.0,2T3Y,2T3Y,32119.27,31499.65,31358.20,60904.0,0.45,-1.93,89.62,91.55,1.12,-2.0
35,442.60,SUNTV.NS,715.0,443.25,446.75,68.0,V40N,ATH,30379.00,30096.80,30141.00,48620.0,-0.15,-0.93,60.04,60.97,1.06,-1.0
39,315.60,VAIBHAVGBL.NS,966.0,313.45,317.48,144.0,2T3Y,2T3Y,45717.12,45446.40,45136.80,139104.0,0.69,-0.59,204.27,204.86,1.60,-1.0
1,1383.30,AAVAS.NS,3196.0,1364.20,1390.60,23.0,2T3Y,2T3Y,31983.80,31815.90,31376.60,73508.0,1.40,-0.52,129.83,130.35,1.12,-1.0
